<a href="https://colab.research.google.com/github/Nskoushik26276/AI-Prompt-Generator/blob/main/AI_Prompt_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import json
import textwrap

USE_OPENAI = False
try:
    from openai import OpenAI
    if os.getenv("OPENAI_API_KEY"):
        client = OpenAI()
        USE_OPENAI = True
    else:
        client = None
except Exception:
    client = None
    USE_OPENAI = False

def sep():
    print("\n" + "-" * 70 + "\n")

def local_refine_intent(raw_text):
    txt = raw_text.strip()
    objective = txt[0].upper() + txt[1:]
    goals = []
    if any(w in txt.lower() for w in ["image", "picture", "photo", "visual", "design"]):
        goals.append("Generate an image or visual asset")
    if any(w in txt.lower() for w in ["code", "script", "program", "app", "website", "tool"]):
        goals.append("Produce runnable code or a software prototype")
    if any(w in txt.lower() for w in ["summary", "explain", "brief", "notes"]):
        goals.append("Summarize or explain information")
    if not goals:
        goals.append("Produce a clear instruction or output that accomplishes the user's request")
    style = "concise, clear, and actionable"
    constraints = "be explicit about inputs, outputs, assumptions, and steps."
    return {"objective": objective, "goals": goals, "style": style, "constraints": constraints}

DEFAULT_CLARIFY_QS = [
    "What is your desired output format? (image, code, text, JSON, etc.)",
    "Any preferred style or tone?",
    "Any constraints or required inputs?"
]

def ask_clarifying_questions(max_n=3):
    print("I'll ask up to", max_n, "clarifying questions. (press Enter to skip)")
    answers = []
    for i in range(max_n):
        q = DEFAULT_CLARIFY_QS[i] if i < len(DEFAULT_CLARIFY_QS) else f"Clarify point {i+1}:"
        a = input(f"Q{i+1}. {q}\nA: ").strip()
        answers.append(a)
    return answers

def gen_chatgpt_prompt(refined, clar_answers):
    objective = refined["objective"]
    goals = "; ".join(refined["goals"])
    style = refined["style"]
    constraints = refined["constraints"]
    clar_text = "\n".join([f"- {a}" for a in clar_answers if a])
    prompt = textwrap.dedent(f"""
    You are an expert assistant. Task: {objective}

    High-level goals:
    {goals}

    Additional clarifications:
    {clar_text if clar_text else '- none'}

    Requirements:
    - Style: {style}
    - Constraints: {constraints}

    Provide:
    1) A concise plan (3-6 steps).
    2) Example inputs and outputs.
    3) A final ready-to-use prompt for another LLM.
    """).strip()
    return prompt

def gen_midjourney_prompt(refined, clar_answers):
    objective = refined["objective"]
    clar_text = " ".join([a for a in clar_answers if a])
    style_hint = clar_answers[1] if len(clar_answers) > 1 and clar_answers[1] else "photorealistic"
    extra = clar_answers[2] if len(clar_answers) > 2 and clar_answers[2] else ""
    prompt = f"{objective}, {clar_text}. {style_hint}, highly detailed, ultra-realistic, cinematic lighting, 8k --v 5 --ar 16:9 {extra}"
    return " ".join(prompt.split())

def gen_stable_diffusion_prompt(refined, clar_answers):
    positive = gen_midjourney_prompt(refined, clar_answers)
    negative = "lowres, blurry, deformed, bad anatomy, watermark, text"
    return {"prompt": positive, "negative_prompt": negative, "sampler": "DPM++ 2M Karras", "cfg_scale": 7.0, "steps": 28}

def gen_code_prompt(refined, clar_answers):
    objective = refined["objective"]
    clar_text = " ".join([a for a in clar_answers if a])
    prompt = textwrap.dedent(f"""
    You are an expert developer. Objective: {objective}
    Clarifications: {clar_text if clar_text else 'none'}

    Provide:
    1) Runnable implementation.
    2) Installation steps.
    3) Example input/output.
    4) Short explanation.
    """).strip()
    return prompt

def gen_json_spec(refined, clar_answers):
    return {
        "objective": refined["objective"],
        "goals": refined["goals"],
        "style": refined["style"],
        "constraints": refined["constraints"],
        "clarifications": [a for a in clar_answers if a]
    }

def openai_refine(prompt):
    if not USE_OPENAI:
        return None
    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Refine clarity and quality."},
                {"role": "user", "content": prompt}
            ]
        )
        return resp.choices[0].message["content"]
    except:
        return None

def main():
    print("\n=== AI Intent Translator ===")
    raw = input("👉 What do you want to do?\n> ").strip()
    if not raw:
        print("No input given.")
        return

    refined = local_refine_intent(raw)
    sep()
    print("Refined interpretation:")
    print(json.dumps(refined, indent=2))
    sep()

    clar = ask_clarifying_questions()

    chat_prompt = gen_chatgpt_prompt(refined, clar)
    mj_prompt = gen_midjourney_prompt(refined, clar)
    sd_spec = gen_stable_diffusion_prompt(refined, clar)
    code_prompt = gen_code_prompt(refined, clar)
    json_spec = gen_json_spec(refined, clar)

    if USE_OPENAI:
        r = input("Refine prompts with OpenAI? (y/N): ").strip().lower()
        if r == "y":
            chat_prompt = openai_refine(chat_prompt) or chat_prompt
            mj_prompt = openai_refine(mj_prompt) or mj_prompt
            code_prompt = openai_refine(code_prompt) or code_prompt

    sep()
    print("Choose output:")
    print("1) ChatGPT prompt")
    print("2) Midjourney prompt")
    print("3) Stable Diffusion JSON")
    print("4) Code generation prompt")
    print("5) Structured JSON spec")
    print("6) All")
    choice = input("Your choice (default 6): ").strip() or "6"
    choice = [c.strip() for c in choice.split(",")]

    if "1" in choice or "6" in choice:
        sep()
        print(chat_prompt)
    if "2" in choice or "6" in choice:
        sep()
        print(mj_prompt)
    if "3" in choice or "6" in choice:
        sep()
        print(json.dumps(sd_spec, indent=2))
    if "4" in choice or "6" in choice:
        sep()
        print(code_prompt)
    if "5" in choice or "6" in choice:
        sep()
        print(json.dumps(json_spec, indent=2))

    sep()
    print("Done.")

if __name__ == "__main__":
    main()



=== AI Intent Translator ===
👉 What do you want to do?
> i want a picture of rajinikanth wearing a dhoti and standing near kasi ganaga ghat

----------------------------------------------------------------------

Refined interpretation:
{
  "objective": "I want a picture of rajinikanth wearing a dhoti and standing near kasi ganaga ghat",
  "goals": [
    "Generate an image or visual asset"
  ],
  "style": "concise, clear, and actionable",
  "constraints": "be explicit about inputs, outputs, assumptions, and steps."
}

----------------------------------------------------------------------

I'll ask up to 3 clarifying questions. (press Enter to skip)
Q1. What is your desired output format? (image, code, text, JSON, etc.)
A: image
Q2. Any preferred style or tone?
A: divine
Q3. Any constraints or required inputs?
A: no

----------------------------------------------------------------------

Choose output:
1) ChatGPT prompt
2) Midjourney prompt
3) Stable Diffusion JSON
4) Code generation p

In [7]:
import os
import json
import textwrap

USE_OPENAI = False
try:
    from openai import OpenAI
    if os.getenv("OPENAI_API_KEY"):
        client = OpenAI()
        USE_OPENAI = True
    else:
        client = None
except Exception:
    client = None
    USE_OPENAI = False

def sep():
    print("\n" + "-" * 70 + "\n")

def local_refine_intent(raw_text):
    txt = raw_text.strip()
    objective = txt[0].upper() + txt[1:]
    goals = []
    if any(w in txt.lower() for w in ["image", "picture", "photo", "visual", "design"]):
        goals.append("Generate an image or visual asset")
    if any(w in txt.lower() for w in ["code", "script", "program", "app", "website", "tool"]):
        goals.append("Produce runnable code or a software prototype")
    if any(w in txt.lower() for w in ["summary", "explain", "brief", "notes"]):
        goals.append("Summarize or explain information")
    if not goals:
        goals.append("Produce a clear instruction or output that accomplishes the user's request")
    style = "concise, clear, and actionable"
    constraints = "be explicit about inputs, outputs, assumptions, and steps."
    return {"objective": objective, "goals": goals, "style": style, "constraints": constraints}

DEFAULT_CLARIFY_QS = [
    "What is your desired output format? (image, code, text, JSON, etc.)",
    "Any preferred style or tone?",
    "Any constraints or required inputs?"
]

def ask_clarifying_questions(max_n=3):
    print("I'll ask up to", max_n, "clarifying questions. (press Enter to skip)")
    answers = []
    for i in range(max_n):
        q = DEFAULT_CLARIFY_QS[i]
        a = input(f"Q{i+1}. {q}\nA: ").strip()
        answers.append(a)
    return answers

def gen_chatgpt_prompt(refined, clar_answers):
    objective = refined["objective"]
    goals = "; ".join(refined["goals"])
    style = refined["style"]
    constraints = refined["constraints"]
    clar_text = "\n".join([f"- {a}" for a in clar_answers if a])
    return textwrap.dedent(f"""
    You are an expert assistant. Task: {objective}

    High-level goals:
    {goals}

    Additional clarifications:
    {clar_text if clar_text else '- none'}

    Requirements:
    - Style: {style}
    - Constraints: {constraints}

    Provide:
    1) A concise plan (3-6 steps).
    2) Example inputs and outputs.
    3) A final ready-to-use prompt for another LLM.
    """).strip()

def gen_midjourney_prompt(refined, clar_answers):
    objective = refined["objective"]
    style_hint = clar_answers[1] if len(clar_answers) > 1 and clar_answers[1] else "photorealistic"
    extra = clar_answers[2] if len(clar_answers) > 2 and clar_answers[2] else ""
    clar_text = " ".join([a for a in clar_answers if a])
    prompt = f"{objective}, {clar_text}. {style_hint}, ultra-detailed, cinematic lighting, 8k --v 5 --ar 16:9 {extra}"
    return " ".join(prompt.split())

def gen_stable_diffusion_prompt(refined, clar_answers):
    positive = gen_midjourney_prompt(refined, clar_answers)
    negative = "lowres, blurry, deformed, watermark, text"
    return {"prompt": positive, "negative_prompt": negative, "sampler": "DPM++ 2M Karras", "cfg_scale": 7.0, "steps": 28}

def gen_code_prompt(refined, clar_answers):
    objective = refined["objective"]
    clar_text = " ".join([a for a in clar_answers if a])
    return textwrap.dedent(f"""
    You are an expert developer. Objective: {objective}
    Clarifications: {clar_text if clar_text else 'none'}

    Provide:
    1) Runnable implementation.
    2) Installation steps.
    3) Example input/output.
    4) Short explanation.
    """).strip()

def gen_json_spec(refined, clar_answers):
    return {
        "objective": refined["objective"],
        "goals": refined["goals"],
        "style": refined["style"],
        "constraints": refined["constraints"],
        "clarifications": [a for a in clar_answers if a]
    }

def gen_final_prompt(refined, clar_answers):
    fmt = clar_answers[0].lower() if clar_answers[0] else ""
    tone = clar_answers[1] if clar_answers[1] else "concise and clear"
    obj = refined["objective"]

    if "image" in fmt:
        return textwrap.dedent(f"""
        Ultra-realistic, {tone} cinematic scene of {obj}.
        Include environmental and atmospheric depth, cultural or contextual elements, high-resolution realism, cinematic lighting, immersive composition, and visually expressive details.
        Ensure the main subject is central and the scene feels rich and complete.
        Output a polished prompt ready for any AI image model.
        """).strip()

    elif "code" in fmt:
        return textwrap.dedent(f"""
        Clear, runnable code for: {obj}.
        Include step-by-step instructions, example input/output, and concise explanation.
        Tone/style: {tone}.
        Output a polished prompt ready for any code-generating AI model.
        """).strip()

    elif "json" in fmt:
        return textwrap.dedent(f"""
        Structured JSON spec for: {obj}.
        Include objectives, goals, style, constraints, and clarifications.
        Output a polished JSON-ready prompt suitable for AI processing.
        """).strip()

    else:
        return textwrap.dedent(f"""
        Clear, actionable text for: {obj}.
        Ensure tone is {tone}.
        Include step-by-step guidance or explanation as appropriate.
        Output a polished prompt ready for any AI text-based model.
        """).strip()

def main():
    print("\n=== AI Intent Translator ===")
    raw = input("👉 What do you want to do?\n> ").strip()
    if not raw:
        print("No input given.")
        return

    refined = local_refine_intent(raw)
    sep()
    print("Refined interpretation:")
    print(json.dumps(refined, indent=2))
    sep()

    clar = ask_clarifying_questions()

    chat_prompt = gen_chatgpt_prompt(refined, clar)
    mj_prompt = gen_midjourney_prompt(refined, clar)
    sd_spec = gen_stable_diffusion_prompt(refined, clar)
    code_prompt = gen_code_prompt(refined, clar)
    json_spec = gen_json_spec(refined, clar)
    final_prompt = gen_final_prompt(refined, clar)

    sep()
    print("Choose output:")
    print("1) ChatGPT prompt")
    print("2) Midjourney prompt")
    print("3) Stable Diffusion JSON")
    print("4) Code generation prompt")
    print("5) Structured JSON spec")
    print("6) All")
    print("7) Final polished prompt")
    choice = input("Your choice (default 6): ").strip() or "6"
    choice = [c.strip() for c in choice.split(",")]

    if "1" in choice or "6" in choice:
        sep()
        print(chat_prompt)
    if "2" in choice or "6" in choice:
        sep()
        print(mj_prompt)
    if "3" in choice or "6" in choice:
        sep()
        print(json.dumps(sd_spec, indent=2))
    if "4" in choice or "6" in choice:
        sep()
        print(code_prompt)
    if "5" in choice or "6" in choice:
        sep()
        print(json.dumps(json_spec, indent=2))

    sep()
    print("Final polished prompt:")
    print(final_prompt)
    sep()
    print("Done.")

if __name__ == "__main__":
    main()



=== AI Intent Translator ===
👉 What do you want to do?
> i want an image in which a man wearing a dhoti stands near kasi ganga ghat

----------------------------------------------------------------------

Refined interpretation:
{
  "objective": "I want an image in which a man wearing a dhoti stands near kasi ganga ghat",
  "goals": [
    "Generate an image or visual asset"
  ],
  "style": "concise, clear, and actionable",
  "constraints": "be explicit about inputs, outputs, assumptions, and steps."
}

----------------------------------------------------------------------

I'll ask up to 3 clarifying questions. (press Enter to skip)
Q1. What is your desired output format? (image, code, text, JSON, etc.)
A: image
Q2. Any preferred style or tone?
A: divine
Q3. Any constraints or required inputs?
A: no

----------------------------------------------------------------------

Choose output:
1) ChatGPT prompt
2) Midjourney prompt
3) Stable Diffusion JSON
4) Code generation prompt
5) Structu